## Test Mutations 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"
os.environ["TOKENIZERS_PARALLELISM"]="false"
os.environ["AWS_REGION_NAME"] = 'us-east-1'

In [3]:
import torch
torch.multiprocessing.set_start_method('spawn')

In [4]:
## setup a test population
from src.utils.script_utils import get_datasets

src.cross_encoder_model.my_evaluator


In [25]:
import pandas as pd

In [26]:
org_data_train, org_data_test, org_data_val = get_datasets("ragtruth", group="QA")
querymode = "queryonly"
#org_data_train.df = perform_query_integration(org_data_train.df, mode=querymode)
#org_data_test.df = perform_query_integration(org_data_test.df, mode=querymode)

955
980
969


In [8]:
from transformers import AutoTokenizer
tok = AutoTokenizer.from_pretrained("facebook/bart-large-mnli")

In [18]:
org_data_train.df
lenlist = []
for r, b in org_data_train.df.iterrows():
    x = tok(b["evidence"], b["claim"], return_tensors='pt')
    print(x.input_ids.shape)
    lenlist.append(x["input_ids"].shape[1])

torch.Size([1, 582])
torch.Size([1, 501])
torch.Size([1, 624])
torch.Size([1, 604])
torch.Size([1, 592])
torch.Size([1, 598])
torch.Size([1, 665])
torch.Size([1, 588])
torch.Size([1, 693])
torch.Size([1, 699])
torch.Size([1, 659])
torch.Size([1, 666])
torch.Size([1, 483])
torch.Size([1, 444])
torch.Size([1, 519])
torch.Size([1, 509])
torch.Size([1, 575])
torch.Size([1, 503])
torch.Size([1, 592])
torch.Size([1, 582])
torch.Size([1, 610])
torch.Size([1, 709])
torch.Size([1, 665])
torch.Size([1, 633])
torch.Size([1, 742])
torch.Size([1, 732])
torch.Size([1, 762])
torch.Size([1, 816])
torch.Size([1, 855])
torch.Size([1, 1024])
torch.Size([1, 513])
torch.Size([1, 594])
torch.Size([1, 590])
torch.Size([1, 752])
torch.Size([1, 725])
torch.Size([1, 766])
torch.Size([1, 476])
torch.Size([1, 438])
torch.Size([1, 391])
torch.Size([1, 760])
torch.Size([1, 586])
torch.Size([1, 519])
torch.Size([1, 377])
torch.Size([1, 273])
torch.Size([1, 381])
torch.Size([1, 402])
torch.Size([1, 432])
torch.Size([

In [14]:
max(lenlist)

1047

In [69]:
org_data_test

In [70]:
## Select a subset of the df for testing
ev_use = org_data_train.df.evidence.unique()
org_data_train.df = org_data_train.df[org_data_train.df.evidence.isin(ev_use)]

In [71]:
from src.utils.script_utils import init_population_from_df

In [72]:
from src.sync_data.initial_generators import FromFile
my_generator = FromFile(type="ClaudeFewShot", path="sync_data", dataset="ragtruth", group="QA", entail_model="vectara_v2", generation_seed=3)
generated_data_population2 = my_generator.generate_samples(org_data_train, 4)

629


You are using a model of type HHEMv2Config to instantiate a model of type HHEMv2. This is not supported for all configurations of models and can yield errors.
/home/ubuntu/DomainAdaptationForFactualConsistencyDetection/SyntheticData/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Computing initial certainties: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1258/1258 [01:25<00:00, 14.68it/s]


In [11]:
tag_list = [[], []]
for t in generated_data_population2.tags: # tasksource
    tag_list[t[1]].append(generated_data_population2.get_initial_prob(t).mean())

In [12]:
import numpy as np
np.array(tag_list).mean(axis=1)

array([0.22523072, 0.8643157 ], dtype=float32)

In [13]:
import numpy as np
np.sum(generated_data_population2.to_dataframe()["sample"].apply(len))

92806

In [39]:
from src.sync_data.pc_mutations import LLMInformationIntroductionMutation, LLMFillInTheGapsMutation

In [40]:
#myllm_intro = LLMInformationIntroductionMutation(n_mutations=3, check_against="evidence", entail_model="vectara_v2")

In [74]:
myllm_intro = LLMFillInTheGapsMutation(device="cuda", n_mutations=3, model_name="claude3-haiku", batch_size=10,
                                        temperature=1.0, mask_output_perc=0.2, connected=True, n_multiply=2, n_threads=4,
                                        preserve_meaning=True, entail_model="vectara_v2")

You are using a model of type HHEMv2Config to instantiate a model of type HHEMv2. This is not supported for all configurations of models and can yield errors.


In [75]:
from src.sync_data.mutation_cache import MutationCache

In [76]:
mycache = MutationCache(myllm_intro, "ragtruth-QA")

In [77]:
len(generated_data_population2)

5032

In [65]:
len(input_pop)

960

In [78]:
input_pop = mycache.mutate_all_tags(generated_data_population2)

Scanning cache for mutations: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1258/1258 [06:05<00:00,  3.44it/s]


Cache hits:  21270
Average tag time:  0.11574527332088712
Remaining examples not in cache:  1487
Using 4 threads.


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 964/964 [00:00<00:00, 1162872.90it/s]


PicklingError: Can't pickle <class 'src.sync_data.pc_mutations.LLMFillInTheGapsMutation'>: it's not the same object as src.sync_data.pc_mutations.LLMFillInTheGapsMutation

In [148]:
len(cache_rep)

51306

In [82]:
cache = cache[~np.isnan(cache.rmiss_vectara_v2)]

In [83]:
cache.to_csv("runs/cache/ragtruth-QA/LLMFillInTheGapsMutation.csv", index=False)

In [49]:
for t in input_pop.tags:
    print(input_pop.get_initial_prob(t))

[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
[0.95468986 0.95468986 0.95468986 0.95468986 0.95468986 0.95468986
 0.99093986 0.99093986 0.99093986 0.929454   0.929454   0.929454
 0.9572199  0.9572199  0.9572199 ]
[0.5       0.5       0.5       0.5       0.5       0.5       0.9041848
 0.9041848 0.9041848 0.5       0.5       0.5       0.9551819 0.9551819
 0.9551819]
[0.5        0.5        0.5        0.5        0.5        0.5
 0.20921786 0.20921786 0.20921786 0.5        0.5        0.5
 0.5        0.5        0.5       ]
[0.94174564 0.94174564 0.94174564 0.94174564 0.94174564 0.94174564
 0.9791166  0.9791166  0.9791166  0.9670195  0.9670195  0.9670195
 0.9601874  0.9601874  0.9601874 ]
[0.03666234 0.03666234 0.03666234 0.03666234 0.03666234 0.03666234
 0.01983724 0.01983724 0.01983724 0.00915457 0.00915457 0.00915457
 0.01551685 0.01551685 0.01551685]
[0.928579   0.928579   0.928579   0.928579   0.928579   0.928579
 0.90167356 0.90167356 0.90167356 0.69950414 0.69950414 0.69

In [23]:
input_test = org_data_train.df.iloc[0].evidence
output_test = org_data_train.df.iloc[0].claim

In [12]:
split_sentences(org_data_train.df.iloc[0].evidence)

['Briefly answer the following question:\ntips how to conserve water\nBear in mind that your response should be strictly based on the following three passages:\npassage 1:1 Take shorter showers.',
 '2  Replace you showerhead with an ultra-low-flow version.',
 '3  Some units are available that allow you to cut off the flow without adjusting the water temperature knobs.',
 '4  Use the minimum amount of water needed for a bath by closing the drain first and filling the tub only 1/3 full.\n\npassage 2:Here are 20 water-saving tips to get you going….',
 '1. Shower Bucket.',
 'Instead of letting the water pour down the drain, stick a bucket under the faucet while you wait for your shower water to heat up.',
 'You can use the water for flushing the toilet or watering your plants.',
 '2. Turn off the tap while brushing your teeth.',
 'Water comes out of the average faucet at 2.5 gallons per minute.',
 'Don’t let all that water go down the drain while you brush!',
 'Turn off the faucet after yo

In [14]:
input_pop = myllm_intro.mutate_all_tags(generated_data_population2)

Using 6 threads.


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:00<00:00, 286300.61it/s]


src.cross_encoder_model.my_evaluator
src.cross_encoder_model.my_evaluator
Done at index  0
Done at index  20
Done at index  30
src.cross_encoder_model.my_evaluator
src.cross_encoder_model.my_evaluator
Done at index  10
src.cross_encoder_model.my_evaluator


Compute Entailments.: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 14.07it/s]


In [18]:
tag_list=list(input_pop.tags)

In [19]:
tag_list[1][1]

1

In [45]:
myllm_intro._compute_rmiss(generated_data_population2[tag_list[1]], input_pop[tag_list[1]], input_pop.get_references(tag_list[1]), tag_list[1], show_progress=False)

array([0.9418001 , 0.9213338 , 0.8812796 , 0.9069611 , 0.90696126,
       0.95943314, 0.5971675 , 0.6143679 , 0.7481089 , 0.92129517,
       0.92129517, 0.9678191 ], dtype=float32)

In [28]:
input_pop.get_references(tag_list[0])

array([0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3])

In [ ]:
generated_data_population2[tag_list[0]]

In [25]:
input_pop.get_initial_prob(tag_list[5])

array([0.7161575 , 0.7615583 , 0.7208725 , 0.6382962 , 0.6033293 ,
       0.70730585, 0.88504475, 0.9746387 , 0.8211213 , 0.5       ,
       0.60294116, 0.50430757], dtype=float32)

In [13]:
from src.sync_data.pc_mutations import LLMFillInTheGapsMutation

In [18]:
llmmutations = LLMFillInTheGapsMutation(device="cuda", n_mutations=3, model_name="claude3-haiku", batch_size=10,
                                        temperature=1.0, mask_output_perc=0.2, connected=True, n_multiply=2, n_threads=4,
                                        preserve_meaning=True, entail_model="tasksource")

In [4]:
from src.sync_data.pc_mutations import LLMSynonymeReplacementMutation

src.cross_encoder_model.my_evaluator


In [5]:
my_synonyms = LLMSynonymeReplacementMutation(device="cuda", n_mutations=3, model_name="claude3-haiku", batch_size=10, temperature=1.0, replace_words=3, n_threads=4, entail_model="tasksource")

In [7]:
input_test = "Conserving water is essential for maintaining water resources and reducing the overall water usage. Here are some tips on how to conserve water based on the given passages:\n\n1. Take shorter showers and replace your showerhead with an ultra-low-flow version. Some units are available that allow you to cut off the flow without adjusting the water temperature knobs.\n2. When taking a bath, fill the tub only 1/3 full instead of using the entire tub."

## Mutation with information introduction deletion

In [8]:
from src.sync_data.compute_entailments import EntailmentCheckModel, split_sentences

In [9]:
split_sentences(input_test)

['Conserving water is essential for maintaining water resources and reducing the overall water usage.',
 'Here are some tips on how to conserve water based on the given passages:\n\n1.',
 'Take shorter showers and replace your showerhead with an ultra-low-flow version.',
 'Some units are available that allow you to cut off the flow without adjusting the water temperature knobs.',
 '2.When taking a bath, fill the tub only 1/3 full instead of using the entire tub.']

In [31]:
from src.sync_data.pc_mutations import DropSentenceMutation

In [42]:
del_mutation = DropSentenceMutation(device="cuda", n_mutations=3, entail_model="tasksource")

In [50]:
input_test = org_data_train.df.iloc[1].evidence
output_test = org_data_train.df.iloc[1].claim

In [51]:
res = del_mutation.mutate([output_test], (input_test, 0))

In [52]:
output_test

'To conserve water, you can take shorter showers, replace your showerhead with an ultra-low-flow version, use the minimum amount of water needed for a bath, and collect shower water in a bucket for other uses. Additionally, you should turn off the tap while brushing your teeth or washing your hands, water lawns in the early morning or late evening in intervals, use a broom instead of a hose to clean driveways and sidewalks, bathe pets outdoors in areas that need water, install a shut-off nozzle on your water hose, and direct downspouts towards shrubs and trees in your garden.'

In [54]:
del_mutation._compute_rmiss([output_test], res[0], res[1], (input_test, 0), show_progress=False)

array([0.5, 0.5, 0.5], dtype=float32)

In [23]:
from src.sync_data.pc_mutations import LLMInformationIntroductionMutation

In [29]:
input_test = org_data_train.df.iloc[0].evidence
output_test = org_data_train.df.iloc[0].claim

In [31]:
output_test

'Here are some tips on how to conserve water: \n\n1. Take shorter showers and consider replacing your showerhead with an ultra-low-flow version. \n2. Use the minimum amount of water needed for a bath by filling the tub only 1/3 full. \n3. Use a bucket to collect the water that runs while waiting for your shower to heat up, this water can then be used for flushing the toilet or watering plants. \n4. Turn off the faucet while brushing your teeth or washing your hands. \n5. Water your lawns in the early morning or later evening in 20-minute intervals. \n6. Use a broom instead of your hose to remove debris from your driveway or sidewalk. \n7. Bathe your pets outdoors in areas that need water. \n8. Install a shut-off nozzle on your water hose to prevent wastage. \n9. Direct downspouts towards shrubs and trees in your garden.'

In [26]:
input_test

'Briefly answer the following question:\ntips how to conserve water\nBear in mind that your response should be strictly based on the following three passages:\npassage 1:1 Take shorter showers. 2  Replace you showerhead with an ultra-low-flow version. 3  Some units are available that allow you to cut off the flow without adjusting the water temperature knobs. 4  Use the minimum amount of water needed for a bath by closing the drain first and filling the tub only 1/3 full.\n\npassage 2:Here are 20 water-saving tips to get you going…. 1. Shower Bucket. Instead of letting the water pour down the drain, stick a bucket under the faucet while you wait for your shower water to heat up. You can use the water for flushing the toilet or watering your plants. 2. Turn off the tap while brushing your teeth. Water comes out of the average faucet at 2.5 gallons per minute. Don’t let all that water go down the drain while you brush! Turn off the faucet after you wet your brush, and leave it off until 

In [42]:
myllm_intro = LLMInformationIntroductionMutation()

In [43]:
print(myllm_intro._generate_prompt_request([output_test], (input_test, 0))[0])

 Your task is to add additional information to a paragraph of text. You are given the following paragraph in <paragraph><\paragraph> tags:
<paragraph>Here are some tips on how to conserve water: 

1. Take shorter showers and consider replacing your showerhead with an ultra-low-flow version. 
2. Use the minimum amount of water needed for a bath by filling the tub only 1/3 full. 
3. Use a bucket to collect the water that runs while waiting for your shower to heat up, this water can then be used for flushing the toilet or watering plants. 
4. Turn off the faucet while brushing your teeth or washing your hands. 
5. Water your lawns in the early morning or later evening in 20-minute intervals. 
6. Use a broom instead of your hose to remove debris from your driveway or sidewalk. 
7. Bathe your pets outdoors in areas that need water. 
8. Install a shut-off nozzle on your water hose to prevent wastage. 
9. Direct downspouts towards shrubs and trees in your garden.</paragraph>
Now you are given

In [49]:
mutations = myllm_intro.mutate([output_test], (input_test, 0))

Done at index  0


In [52]:
print(output_test)

Here are some tips on how to conserve water: 

1. Take shorter showers and consider replacing your showerhead with an ultra-low-flow version. 
2. Use the minimum amount of water needed for a bath by filling the tub only 1/3 full. 
3. Use a bucket to collect the water that runs while waiting for your shower to heat up, this water can then be used for flushing the toilet or watering plants. 
4. Turn off the faucet while brushing your teeth or washing your hands. 
5. Water your lawns in the early morning or later evening in 20-minute intervals. 
6. Use a broom instead of your hose to remove debris from your driveway or sidewalk. 
7. Bathe your pets outdoors in areas that need water. 
8. Install a shut-off nozzle on your water hose to prevent wastage. 
9. Direct downspouts towards shrubs and trees in your garden.


In [54]:
print(mutations[0][2])

Here are some tips on how to conserve water:

1. Take shorter showers and consider replacing your showerhead with an ultra-low-flow version.
2. Use the minimum amount of water needed for a bath by filling the tub only 1/3 full.
3. Use a bucket to collect the water that runs while waiting for your shower to heat up, this water can then be used for flushing the toilet or watering plants. Instead of letting the water pour down the drain, stick a bucket under the faucet while you wait for your shower water to heat up.
4. Turn off the faucet while brushing your teeth or washing your hands.
5. Water your lawns in the early morning or later evening in 20-minute intervals.
6. Use a broom instead of your hose to remove debris from your driveway or sidewalk.
7. Bathe your pets outdoors in areas that need water.
8. Install a shut-off nozzle on your water hose to prevent wastage.
9. Direct downspouts towards shrubs and trees in your garden.


In [40]:
req = my_synonyms._generate_prompt_request([input_test], None)

NameError: name 'my_synonyms' is not defined

In [41]:
print(req[0])

NameError: name 'req' is not defined

In [9]:
mutations  = my_synonyms.mutate([input_test], None)

Invalid response: Here is the first answer with 3 words replaced by synonyms:

<answer 0>
Conserving water is essential for preserving water resources and diminishing the overall water usage. Here are some tips on how to conserve water based on the given passages:

1. Take briefer showers and substitute your showerhead with an ultra-low-flow version. Some units are available that allow you to interrupt the flow without modifying the water temperature knobs.
2. When taking a bath, fill the tub only 1/3 full instead of utilizing the entire tub.
</answer 0>
invalid response received. Retrying.
Invalid response: Here is the first answer with 3 words replaced by synonyms:

<answer 0>
Conserving water is essential for preserving water resources and diminishing the overall water usage. Here are some tips on how to conserve water based on the given passages:

1. Take briefer showers and substitute your showerhead with an ultra-low-flow version. Some units are available that allow you to interr

In [10]:
mutations

(['Preserving water is essential for sustaining water resources and lowering the comprehensive water usage. Here are some suggestions on how to preserve water based on the given passages:\n\n1. Take briefer showers and swap your showerhead with an ultra-low-flow model. Some units are accessible that permit you to halt the flow without modifying the water temperature controls.\n2. When taking a bath, fill the tub only 1/3 full instead of utilizing the whole tub.',
  'Conserving water is crucial for upholding water resources and decreasing the overall water usage. Here are some tips on how to conserve water based on the given passages:\n\n1. Take shorter showers and swap your showerhead with an ultra-low-flow version. Some units are available that enable you to cut off the flow without adjusting the water temperature knobs.\n2. When taking a bath, fill the tub only 1/3 full instead of using the complete tub.',
  'Conserving water is essential for maintaining water resources and reducing 

In [ ]:
## Test LLM information insertion mutation.
input_

In [20]:
cached_mutation = MutationCache(llmmutations, "ragtruth-QA")

In [ ]:
popu_mutate = cached_mutation.mutate_all_tags(generated_data_population2)

Scanning cache for mutations: 100%|████████████████████████████████████████████████████████████████████████████| 1258/1258 [00:05<00:00, 225.25it/s]


Cache hits:  342
Average tag time:  0.1841007157375938
Remaining examples not in cache:  4975
Using 4 threads.


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 1251/1251 [00:00<00:00, 692043.56it/s]


In [21]:
popu_mutate.to_dataframe().to_csv("temp/mutation_llm.csv")

In [106]:
from src.sync_data.pc_mutations import RephraseMutation

In [112]:
mymutation = RephraseMutation(n_mutations=1)

In [113]:
updated_popu = mymutation.mutate_all_tags(generated_data_population2)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [01:10<00:00,  1.75s/it]


In [8]:
from src.sync_data.pc_mutations import WordReplacementMutation

In [62]:
from src.sync_data.pc_mutations import WordInsertionMutation

In [71]:
replace_mutation = WordReplacementMutation(n_mutations=3, n_multiply=3, consider_top_k=5, device="cuda:0", entail_model=None, model_str="FacebookAI/roberta-large", tokenizer_str="FacebookAI/roberta-large", rerank_similarity=False)

In [87]:
replace_mutation = WordReplacementMutation(n_mutations=3, n_multiply=3, consider_top_k=5, device="cuda:0", entail_model=None, model_str="distilbert/distilroberta-base", tokenizer_str="distilbert/distilroberta-base", rerank_similarity=True)

Some weights of the model checkpoint at distilbert/distilroberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [102]:
#replace_mutation = WordInsertionMutation(n_mutations=3, n_multiply=3, consider_top_k=5, device="cuda:0", entail_model=None, model_str="FacebookAI/roberta-large", tokenizer_str="FacebookAI/roberta-large", rerank_similarity=False)
replace_mutation = WordInsertionMutation(n_mutations=3, n_multiply=3, consider_top_k=5, device="cuda:0", entail_model=None, model_str="google-t5/t5-base", tokenizer_str="google-t5/t5-base", rerank_similarity=True)

In [104]:
resp = replace_mutation.mutate_all_tags(generated_data_population2)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:03<00:00, 10.33it/s]


In [105]:
len(resp)

1440

In [55]:
res

['John is walking through the park.',
 'I enjoy going long strolls in New York City.',
 'John is walking through the park,',
 'I enjoy going short strolls in New York City.',
 'John is walking through the park:',
 'I enjoy going on strolls in New York City.',
 'John is wandering through the park.',
 'I enjoy going for strolls around New York City.',
 'John is walking through the park.',
 'I enjoy going for strolls through New York City.',
 'John is driving through the park.',
 'I enjoy going for strolls throughout New York City.',
 'John is walking through the rain.',
 'I enjoy going for strolls in New Mexico City.',
 'John is walking through the forest.',
 'I enjoy going for strolls in New Orleans City.',
 'John is walking through the woods.',
 'I enjoy going for strolls in New York City.']

In [44]:
res

['John is walking through the architecture.',
 'I enjoy there for strolls in New York City.',
 'John is walking through the fork.',
 'I enjoy seasons for strolls in New York City.',
 'John is walking through the lights.',
 'I enjoy Chinatown for strolls in New York City.',
 'John is half through the park.',
 'I enjoy going for strolls in - York City.',
 'John is approaching through the park.',
 'I enjoy going for strolls in uniform York City.',
 'John is amplified through the park.',
 'I enjoy going for strolls in Tropical York City.',
 'John Westbrook walking through the park.',
 'I enjoy going for strolls in Total York City.',
 'John Gustav walking through the park.',
 'I enjoy going for strolls in lifelong York City.',
 'John Branch walking through the park.',
 'I enjoy going for strolls in Gothic York City.']

In [49]:
import gensim.downloader

In [50]:
glove_vectors = gensim.downloader.load('glove-twitter-25')

[===========---------------------------------------] 22.4% 23.4/104.8MB downloaded

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



[==================================----------------] 69.4% 72.7/104.8MB downloaded

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



[==================================================] 100.0% 104.8/104.8MB downloaded


In [35]:
len(generated_data_population2)

160

In [39]:
from src.sync_data.mutation_cache import MutationCache

In [37]:
cached_mutation = MutationCache(replace_mutation, "cache_2")

NameError: name 'MutationCache' is not defined

In [17]:
updated_popu_repl_full = replace_mutation.mutate_all_tags(generated_data_population2)

Compute Entailments.: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 24.64it/s]

Compute Entailments.: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 21.14it/s]

Compute Entailments.: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

In [115]:
updated_popu_repl.to_dataframe().to_csv("runs/cache_test_2/first_part.csv", index=False)

In [116]:
len(updated_popu_repl_full[tag_test])

18

In [117]:
len(test_population[tag_test])

5

In [118]:
## create new mutation using the cache
cached_mutation = MutationCache(replace_mutation, "cache_test_2")

In [15]:
updated_popu_repl_all = cached_mutation.mutate_all_tags(test_population)

NameError: name 'cached_mutation' is not defined

In [111]:
updated_popu_repl_full.get_references(tag_test)

array([0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1])

In [168]:
backrefs = updated_popu_repl_full.get_agreement(tag_test)

In [169]:
backrefs.mean()

0.6950257652335696

In [172]:
updated_popu_repl_full.get_agreement(tag_test)

array([0.76185989, 0.77742565, 0.70900398, 0.77742565, 0.5       ,
       0.78329188, 0.65586376, 0.77742565, 0.62978369, 0.57675678,
       0.75124961, 0.5       , 0.71313429, 0.70280814, 0.59353119,
       0.77032661, 0.78399187, 0.74658513])

In [20]:
arr = np.array([4, 27, 6])
np.median(arr)

6.0

In [101]:
i=4
print(test_population[tag_test][i])

Sure, here are some tips for conserving water based on the given passages:

1. Take shorter showers and replace your showerhead with an ultra-low-flow version. (Passage 1)
2. Use a bucket to collect water while waiting for your shower water to heat up and use it for flushing the toilet or watering plants. (Passage 2)
3. Turn off the tap while brushing your teeth and washing your hands. (Passage 2)
4. Water your lawns in the early morning or late evening in 20-minute intervals. (Passage 3)
5. Use a broom instead of a hose to remove debris from your driveway or sidewalk. (Passage 3)
6. Bathe your pets outdoors in areas that need water. (Passage 3)
7. Install a shut-off nozzle on your water hose to prevent wasting water. (Passage 3)
8. Direct downspouts towards shrubs and trees in your garden. (Passage 3)

I hope these tips help you conserve water!


In [102]:
for sample in updated_popu_repl_all[tag_test][backrefs==i]:
    print(sample)

Sure, here are some tips for conserving water based on the given passages:

1. Take shorter showers and replace your showerhead with an ultra-low-flow version. (Passage 1)
2. Use a bucket to collect water while waiting for your shower water to heat up and use it for flushing the toilet or watering plants. (Passage 2)
3. Turn off the tap while brushing your teeth and washing your hands. (Passage 2)
4. Water your lawns in the early morning or late evening in 20-minute intervals. (Passage 3)
5. Use a broom instead of a hose to remove debris from your driveway or sidewalk. (Passage 3)
6. Bathe your pets outdoors in areas that need water. (Passage 3)
7. Install a shut-off nozzle on your water hose to prevent wasting water. (Passage 3)
8. Direct downspouts towards shrubs and trees in your garden. (Passage 3)

I hope these tips help you conserve water!
Sure, here are some tips for conserving water based on the given passages:

1. Take shorter showers and replace your showerhead with an ultra-

In [23]:
len(test_population.to_dataframe()["sample"].unique())

119

In [88]:
len(updated_popu_repl_all)

1620

In [28]:
all_popu = test_population + updated_popu_repl + updated_popu + popu_mutate

join
join
join


In [30]:
len(all_popu)

2040

In [46]:
updated_popu_repl.to_dataframe().iloc[0]["sample"]

'Conserving water is essential for maintaining water resources and reducing the overall water usage. Here are some tips on how to conserve water based on the given passages:\n\n1. Take shorter showers and replace your showerhead with an ultra-low-flow version. Some units are available that allow you to cut off the flow without adjusting the water temperature knobs.\n2. When taking a bath, fill the tub only 1/3 full instead of using the entire tub.\n3. Use a shower bucket instead of letting the water pour down the drain while you wait for your shower water to heat up. You can use the water for flushing the toilet or watering your plants.\n4. Turn off the tap while brushing your hair and when washing your hands.\n5. Water your lawns in the early morning or later evening in 20 minute intervals. Use a broom instead of a hose to remove debris from your driveway or sidewalk.\n6. Bathe your pets outdoors in areas that need water.\n7. Install a shut-off nozzle on your water hose; a garden hose

In [48]:
updated_popu_repl.to_dataframe().iloc[1]["sample"]

'Conserving water is essential for maintaining water resources and reducing the overall water usage. Here are some tips on how to conserve water based on the given passages:\n\n1. Take shorter showers and replace your showerhead with an ultra-low-flow version. Some units are available that allow you to cut off the flow without adjusting the water temperature knobs.\n2. When taking a bath, fill the tub only 1/3 full instead of using the entire tub.\n3. Use a shower bucket instead of letting the water pour down the drain while you wait for your shower water to heat up. You can use the water for flushing the toilet or watering your plants.\n4. Turn off the tap while brushing your tooth and when washing your hands.\n5. Water your lawns in the early morning or later evening in 20 minute intervals. Use a broom instead of a hose to remove debris from your driveway or sidewalk.\n6. Bathe your pets outdoors in areas that need water.\n7. Install a shut-off nozzle on your water hose; a garden hos

In [47]:
test_population.to_dataframe().iloc[0]["sample"]

'Conserving water is essential for maintaining water resources and reducing the overall water usage. Here are some tips on how to conserve water based on the given passages:\n\n1. Take shorter showers and replace your showerhead with an ultra-low-flow version. Some units are available that allow you to cut off the flow without adjusting the water temperature knobs.\n2. When taking a bath, fill the tub only 1/3 full instead of using the entire tub.\n3. Use a shower bucket instead of letting the water pour down the drain while you wait for your shower water to heat up. You can use the water for flushing the toilet or watering your plants.\n4. Turn off the tap while brushing your teeth and when washing your hands.\n5. Water your lawns in the early morning or later evening in 20 minute intervals. Use a broom instead of a hose to remove debris from your driveway or sidewalk.\n6. Bathe your pets outdoors in areas that need water.\n7. Install a shut-off nozzle on your water hose; a garden hos

In [50]:
updated_popu_repl.to_dataframe().iloc[0]["sample"] == test_population.to_dataframe().iloc[0]["sample"]

False

In [2]:
import pandas as pd
cache = pd.read_csv("runs/cache/ragtruth-QA/LLMFillInTheGapsMutation.csv")

In [3]:
len(cache)

77318

In [12]:
cnts = cache.groupby("old_claim").count()

In [13]:
cnts.describe()

,tag_0,tag_1,new_claim,used
count,5778.000000,5778.000000,5778.000000,5778.000000
mean,13.381447,13.381447,13.381447,13.381447
std,16.351838,16.351838,16.351838,16.351838
min,3.000000,3.000000,3.000000,3.000000
25%,10.000000,10.000000,10.000000,10.000000
50%,14.000000,14.000000,14.000000,14.000000
75%,16.000000,16.000000,16.000000,16.000000
max,1207.000000,1207.000000,1207.000000,1207.000000


In [118]:
from src.sync_data.pc_mutations import BackTranslationMutation

In [119]:
llmbacktrans = BackTranslationMutation(n_mutations=3)

Starting to load English to German Translation Model.



Some weights of FSMTForConditionalGeneration were not initialized from the model checkpoint at facebook/wmt19-en-de and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Completed loading English to German Translation Model.

Starting to load German to English Translation Model:


Some weights of FSMTForConditionalGeneration were not initialized from the model checkpoint at facebook/wmt19-de-en and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Completed loading German to English Translation Model.



In [120]:
popu_new = llmbacktrans.mutate_all_tags(generated_data_population2)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [05:02<00:00,  7.56s/it]


In [121]:
len(popu_new)

480

In [122]:
len(generated_data_population2)

160

In [123]:
popu_new.add_to_refs(10000)

In [125]:
tag_use = list(popu_new.tags)[0]

In [126]:
popu_new.get_references(tag_use)

array([10000, 10000, 10000, 10001, 10001, 10001, 10002, 10002, 10002,
       10003, 10003, 10003])

In [3]:
from sentence_transformers import CrossEncoder
model = CrossEncoder('cross-encoder/nli-deberta-v3-base')

/home/ubuntu/DomainAdaptationForFactualConsistencyDetection/SyntheticData/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/ubuntu/DomainAdaptationForFactualConsistencyDetection/SyntheticData/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [16]:
sample = generated_data_population2.to_dataframe().iloc[5]

In [17]:
scores = model.predict([(sample.tag_0, sample["sample"])])

In [18]:
scores

array([[-5.3501353,  4.405088 , -0.5322344]], dtype=float32)

In [7]:
from src.sync_data.compute_entailments import EntailmentCheckModel
emodel = EntailmentCheckModel("tasksource", device="cuda:0")

In [9]:
emodel.reset_inplace()

In [ ]:
OneWayCrossEncoder